<a href="https://colab.research.google.com/github/Frxn5J/GeneracionDeTexto_4bits_Gdrive/blob/main/GeneracionDeTexto_4bits_Gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno de generación de texto LLM para Google Colab

Este cuaderno utiliza https://github.com/oobabooga/text-generation-webui para ejecutar modelos de conversación en modo de chat.

Ejecute todas las celdas y una URL pública de Gradio aparecerá en la parte inferior en unos 5 minutos.

https://status.gradio.app/

## Parámetros

* **save_logs_to_google_drive**: guarda automáticamente sus registros de chat, personajes y softprompts en Google Drive, para que persistan en todas las sesiones.
* **text_streaming**: transmite la salida de texto en tiempo real en lugar de esperar a que se complete toda la respuesta.
* **cai_chat**: hace que la interfaz parezca Character.AI. De lo contrario, parece un chat similar a WhatsApp estándar.
* **load_in_8bit**: carga el modelo con precisión de 8 bits, reduciendo el uso de memoria de la GPU a la mitad. Esto le permite usar la longitud completa de la señal de prompt de 2048 sin quedarse sin memoria, con un pequeño costo de precisión y velocidad.
* **activate_silero_text_to_speech**: las respuestas serán audios en lugar de texto. Hay 118 voces disponibles (en_0 a en_117), que se pueden establecer en la pestaña "Extensiones" de la interfaz. Puede encontrar ejemplos aquí: (https://oobabooga.github.io/silero-samples/).
* **activate_sending_pictures**: agrega un menú para enviar imágenes al bot, que se etiquetan automáticamente con BLIP.
* **activate_character_bias**: una extensión que agrega una cadena oculta definida por el usuario al comienzo de la respuesta del bot con el objetivo de sesgar el resto de la respuesta.
* **chat_language**: si es diferente al inglés, activa la traducción automática usando Google Translate, lo que le permite comunicarse con el bot en un idioma diferente.

## Personajes

Puede usar los siguientes sitios web para crear personajes compatibles con esta interfaz web:

* [JSON character creator](https://oobabooga.github.io/character-creator.html)
* [AI Character Editor](https://zoltanai.github.io/character-editor/)

## Creditos

Basado en el [cuaderno original de 81300](https://colab.research.google.com/github/81300/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb).


In [ ]:
#@title 1. Mantenga esta pestaña activa para evitar que Colab lo desconecte. { display-mode: "form" }

#@markdown Presione el botón de reproducción en el reproductor de música que aparecerá debajo:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Instale la interfaz de usuario web.
#remember gradio is currently held back
save_logs_to_google_drive = False #@param {type:"boolean"} 
save_everything_to_google_drive = True #@param {type:"boolean"} 
#@markdown Recuerde que estos modelos son grandes y la cuenta de Google Drive gratuita solo ofrece 15GB de almacenamiento. <br>
install_gptq = True #@param {type:"boolean"}
#@markdown Instale GPTQ-for-LLaMa para modelos cuantificados de 4 bits que requieran --wbits 4.
from IPython.display import clear_output
if save_logs_to_google_drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount('/content/drive')
  base_folder = '/content/drive/MyDrive'

if save_everything_to_google_drive:
    import os
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    base_folder = '/content/drive/MyDrive'
    repo_dir = '/content/drive/MyDrive/text-generation-webui'
    model_dir = '/content/drive/MyDrive/text-generation-webui/models'
    gptq_dir = '/content/drive/MyDrive/text-generation-webui/repositories/GPTQ-for-LLaMa'
    if os.path.exists(repo_dir):
        %cd {repo_dir}
        !git pull
    else:
        %cd /content/drive/MyDrive/
        !git clone https://github.com/oobabooga/text-generation-webui

else:
    model_dir = '/content/text-generation-webui/models'
    repo_dir = '/content/text-generation-webui'
    %cd /content
    !git clone https://github.com/oobabooga/text-generation-webui



if save_logs_to_google_drive:
  if not os.path.exists(f"{base_folder}/oobabooga-data"):
    os.mkdir(f"{base_folder}/oobabooga-data")
  if not os.path.exists(f"{base_folder}/oobabooga-data/logs"):
    os.mkdir(f"{base_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{base_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{base_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{base_folder}/oobabooga-data/characters"):
    shutil.move("text-generation-webui/characters", f"{base_folder}/oobabooga-data/characters")
  else:
    !rm -r "text-generation-webui/characters"
    
  !rm -r "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/logs" "text-generation-webui/logs"
  !ln -s "$base_folder/oobabooga-data/softprompts" "text-generation-webui/softprompts"
  !ln -s "$base_folder/oobabooga-data/characters" "text-generation-webui/characters"

else:
  !mkdir text-generation-webui/logs

!ln -s text-generation-webui/logs .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/models .
%rm -r sample_data
%cd text-generation-webui
!wget https://raw.githubusercontent.com/pcrii/Philo-Colab-Collection/main/settings-colab-template.json -O settings-colab-template.json

# Install requirements
!pip install -r requirements.txt
!pip install -r extensions/google_translate/requirements.txt
!pip install -r extensions/silero_tts/requirements.txt
print(f"\033[1;32;1m\n --> If you see a warning about \"pydevd_plugins\", just ignore it and move on to Step 3. There is no need to restart the runtime.\n\033[0;37;0m")

if install_gptq:
    if save_everything_to_google_drive:
        if os.path.exists(gptq_dir):
            %cd {gptq_dir}
            !git pull
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install

        else:
            !mkdir /content/drive/MyDrive/text-generation-webui/repositories
            %cd /content/drive/MyDrive/text-generation-webui/repositories
            !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
            %cd GPTQ-for-LLaMa
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install
    else:
        %mkdir /content/text-generation-webui/repositories/
        %cd /content/text-generation-webui/repositories/
        !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
        %cd GPTQ-for-LLaMa
        !pip install ninja
        !pip install -r requirements.txt
        !python setup_cuda.py install
clear_output()
print("Finished")

In [ ]:
#@title Descarga el MODELO
#@markdown Puede insertar cualquier modelo de Hugging Face en el formato Organización/modelo
model_download = "facebook/opt-1.3b" #@param [ "facebook/opt-1.3b", "anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g", "anon8231489123/vicuna-13b-GPTQ-4bit-128g"] {allow-input: true}
#@markdown  Descarga el modelo
%cd {repo_dir}
!python download-model.py {model_download}
#this lists directorys from your model folder you can copy the name provided for the model you want for use in the the next cell
!rm {model_dir}/place-your-models-here.txt
clear_output()
if save_logs_to_google_drive or save_everything_to_google_drive:
    drive_NOT_mounted = False
else:
    drive_NOT_mounted = True

if drive_NOT_mounted:
  import os
print("Available Models")
print(os.listdir(model_dir))



In [ ]:
#@title 3. Launch

import json

#@markdown Si no sabe qué ingresar, la celda anterior debería haber impreso los inputs disponibles.<br> Pegue eso aquí.
model_load = "anon8231489123_vicuna-13b-GPTQ-4bit-128g" #@param {type:"string"}
# Parameters
#auto_devices = False #@param {type:"boolean"}
load_4bit_models = True #@param {type:"boolean"}

groupsize_128 = True #@param {type:"boolean"}
load_in_8bit = False #@param {type:"boolean"}
chat = True #@param {type:"boolean"}

text_streaming = True #@param {type:"boolean"}
activate_silero_text_to_speech = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}
chat_language = "Spanish" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']


activate_google_translate = (chat_language != "English")

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


%cd {repo_dir}
# Applying the selected language and setting the prompt size to 2048
# if 8bit mode is selected
j = json.loads(open('settings-colab-template.json', 'r').read())
j["google_translate-language string"] = language_codes[chat_language]
if load_in_8bit:
  j["chat_prompt_size"] = 2048
with open('settings-colab.json', 'w') as f:
  f.write(json.dumps(j, indent=4))

params = set()
if chat:
  params.add('--cai-chat')

if load_in_8bit:
  params.add('--load-in-8bit')
#if auto_devices:
#  params.add('--auto-devices')
if load_4bit_models:
  params.add('--wbits 4')

if groupsize_128:
  params.add('--groupsize 128')

active_extensions = []
if activate_sending_pictures:
  active_extensions.append('send_pictures')
if activate_character_bias:
  active_extensions.append('character_bias')
if activate_google_translate:
  active_extensions.append('google_translate')
if activate_silero_text_to_speech:
  active_extensions.append('silero_tts')
active_extensions.append('gallery')

if len(active_extensions) > 0:
  params.add(f'--extensions {" ".join(active_extensions)}')

if not text_streaming or activate_google_translate or activate_silero_text_to_speech:
  params.add('--no-stream')
if activate_character_bias:
  params.add('--verbose')

# Starting the web UI
cmd = f"python server.py --share --model {model_load} --settings settings-colab.json {' '.join(params)}"
print(cmd)
!$cmd

In [ ]:
#@title Opcional: instale el archivo de personaje "LLaMa" que encontré en Reddit.
!wget https://github.com/pcrii/Philo-Colab-Collection/raw/main/llama.json 
!mv llama.json {repo_dir}/characters
